In [1]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [2]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

In [3]:
l0 = Dense(units=1, input_shape=[1])
model = Sequential([l0])
model.compile(optimizer='sgd', loss='mean_squared_error')

xs = np.array([-1.0, 0.0, 1.0, 2.0, 3.0, 4.0], dtype=float)
ys = np.array([-3.0, -1.0, 1.0, 3.0, 5.0, 7.0], dtype=float)

model.fit(xs, ys, epochs=500)

print(model.predict([10.0]))
print("Here is what I learned: {}".format(l0.get_weights()))

Epoch 1/500
1/1 [==============================] - 4s 4s/step - loss: 9.0306
Epoch 2/500
1/1 [==============================] - 0s 11ms/step - loss: 7.3094
Epoch 3/500
1/1 [==============================] - 0s 10ms/step - loss: 5.9509
Epoch 4/500
1/1 [==============================] - 0s 9ms/step - loss: 4.8781
Epoch 5/500
1/1 [==============================] - 0s 11ms/step - loss: 4.0300
Epoch 6/500
1/1 [==============================] - 0s 7ms/step - loss: 3.3588
Epoch 7/500
1/1 [==============================] - 0s 11ms/step - loss: 2.8269
Epoch 8/500
1/1 [==============================] - 0s 9ms/step - loss: 2.4046
Epoch 9/500
1/1 [==============================] - 0s 9ms/step - loss: 2.0687
Epoch 10/500
1/1 [==============================] - 0s 9ms/step - loss: 1.8007
Epoch 11/500
1/1 [==============================] - 0s 11ms/step - loss: 1.5864
Epoch 12/500
1/1 [==============================] - 0s 9ms/step - loss: 1.4142
Epoch 13/500
1/1 [==============================] - 0s 9m

In [4]:
export_dir = 'saved_model/1'
tf.saved_model.save(model, export_dir)

In [5]:
# Convert the model.
converter = tf.lite.TFLiteConverter.from_saved_model(export_dir)
tflite_model = converter.convert()

In [6]:
import pathlib
tflite_model_file = pathlib.Path('model.tflite')
tflite_model_file.write_bytes(tflite_model)

1080

In [7]:
# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_content=tflite_model)
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print(input_details)
print(output_details)

[{'name': 'serving_default_dense_input:0', 'index': 0, 'shape': array([1, 1], dtype=int32), 'shape_signature': array([-1,  1], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
[{'name': 'StatefulPartitionedCall:0', 'index': 3, 'shape': array([1, 1], dtype=int32), 'shape_signature': array([-1,  1], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]


In [8]:
to_predict = np.array([[10.0]], dtype=np.float32)
print(to_predict)
interpreter.set_tensor(input_details[0]['index'], to_predict)
interpreter.invoke()
tflite_results = interpreter.get_tensor(output_details[0]['index'])
print(tflite_results)

[[10.]]
[[18.983055]]
